In [9]:
import pandas as pd
from tqdm import tqdm

#### Оставили только рабочие ссылки и получили csv

(полный код в create_new_csv.ipynb, это делали делали в облаке, результат сохранен в data_correct.csv)

In [5]:
correct_links = pd.read_csv("data_correct.csv")
print(correct_links.shape)
correct_links.head()

(1104969, 2)


,path,verdict
0,/home/jupyter/mnt/datasets/pictures/pic_0.jpg,clean_photo_good_background
1,/home/jupyter/mnt/datasets/pictures/pic_1.jpg,clean_photo_good_background
2,/home/jupyter/mnt/datasets/pictures/pic_2.jpg,other_infographics
3,/home/jupyter/mnt/datasets/pictures/pic_3.jpg,clean_photo_good_background
4,/home/jupyter/mnt/datasets/pictures/pic_4.jpg,good_infographics


#### Формирование выборки для бинарной классификации: 

есть ли инфографика или нет инфографики (чистое фото)

In [7]:
label_names = correct_links.verdict.unique()
label_names

array(['clean_photo_good_background', 'other_infographics',
       'good_infographics', 'clean_photo_other_background',
       'bad_infographics', 'clean_photo_bad_background',
       'clean_photo_image_background'], dtype=object)

In [12]:
new_label_name = {
    'clean_photo_good_background': 'clean_photo', 
    'other_infographics': "infographics",
    'good_infographics': "infographics", 
    'clean_photo_other_background': "clean_photo",
    'bad_infographics': "infographics", 
    'clean_photo_bad_background': 'clean_photo',
    'clean_photo_image_background': 'clean_photo'
}
tqdm.pandas()
frame_for_binary = correct_links.copy()
frame_for_binary["verdict"] = frame_for_binary.verdict.progress_apply(lambda x: new_label_name[x])
frame_for_binary.head()

100%|██████████| 1104969/1104969 [00:00<00:00, 1964968.17it/s]


,path,verdict
0,/home/jupyter/mnt/datasets/pictures/pic_0.jpg,clean_photo
1,/home/jupyter/mnt/datasets/pictures/pic_1.jpg,clean_photo
2,/home/jupyter/mnt/datasets/pictures/pic_2.jpg,infographics
3,/home/jupyter/mnt/datasets/pictures/pic_3.jpg,clean_photo
4,/home/jupyter/mnt/datasets/pictures/pic_4.jpg,infographics


In [17]:
frame_for_binary.verdict.value_counts()

infographics    557493
clean_photo     547476
Name: verdict, dtype: int64

In [27]:
pd.concat([
    frame_for_binary[frame_for_binary.verdict == "clean_photo"].sample(30000, random_state=42), 
    frame_for_binary[frame_for_binary.verdict == "infographics"].sample(30000, random_state=42)
]).to_csv("data_for_binary.csv", index=False)

#### Формирование выборки только для инфографики

In [52]:
only_infgr = correct_links.copy()
only_infgr = only_infgr[only_infgr.verdict.isin([
   'other_infographics',
   'good_infographics',
   'bad_infographics'
])]
only_infgr.verdict.value_counts()

good_infographics     292314
other_infographics    215554
bad_infographics       49625
Name: verdict, dtype: int64

In [53]:
name_label_gr = only_infgr.verdict.unique()
csv_only_infgr = pd.DataFrame({"path": [], "verdict": []})
for name in name_label_gr:
    csv_only_infgr = pd.concat([ 
        csv_only_infgr,
        only_infgr[only_infgr.verdict == name].sample(20000, random_state=42)
              ])

In [54]:
print(csv_only_infgr.verdict.value_counts())
csv_only_infgr.to_csv("data_for_infographics.csv", index=False)

other_infographics    20000
good_infographics     20000
bad_infographics      20000
Name: verdict, dtype: int64


#### Формирование выборки только для фона (чистое фото)

In [55]:
only_photo = correct_links.copy()
only_photo = only_photo[only_photo.verdict.isin([
   'clean_photo_good_background',
   'clean_photo_other_background',
   'clean_photo_bad_background',
   'clean_photo_image_background'
])]
only_photo.verdict.value_counts()

clean_photo_good_background     455307
clean_photo_other_background     49569
clean_photo_bad_background       28093
clean_photo_image_background     14507
Name: verdict, dtype: int64

In [56]:
name_label_ph = only_photo.verdict.unique()
csv_only_ph = pd.DataFrame({"path": [], "verdict": []})
for name in name_label_ph:
    csv_only_ph = pd.concat([ 
        csv_only_ph,
        only_photo[only_photo.verdict == name].sample(14507, random_state=42)
              ])

In [51]:
print(csv_only_infgr.verdict.value_counts())
csv_only_ph.to_csv("data_for_background.csv", index=False)

clean_photo_good_background     14507
clean_photo_other_background    14507
clean_photo_bad_background      14507
clean_photo_image_background    14507
Name: verdict, dtype: int64


In [60]:
pd.read_csv("data_for_background.csv").verdict.unique()

array(['clean_photo_good_background', 'clean_photo_other_background',
       'clean_photo_bad_background', 'clean_photo_image_background'],
      dtype=object)

In [61]:
pd.read_csv("data_for_infographics.csv").verdict.unique()

array(['other_infographics', 'good_infographics', 'bad_infographics'],
      dtype=object)